# Demo of functions load\_lss2\_clean & updown\_sample from*./swiss_utils/data_cube_utilities/sdc_utilities.py*

*****

__This script is the "official demo" of a function. Please if you want to modify it, work on your own copy__

Creates a clean dataset mixing Landsat and Sentinel 2 products (respectively with prefixs 'ls' and 's2') and using cleaning "autor's recommended ways":
- ls_qa_clean
- create_slc_clean_mask

Sorted by ascending time.

If `resample` option is activated ('up' or 'down_mean', 'down_median') up/downsampling is performed and products output combined into a single 'lss2' prefix. 

This function works as [demo_FUN_load_multi_clean.ipynb](demo_FUN_load_multi_clean.ipynb) function, but with a mix of Landsat and Sentinel 2 products. The `resampl` option was added (to optionally combine products output).

<mark>!!! This function was developped specifically for the first version of SDC (which only used ingested products) then it will have to be rewritten for the actual version of SDC !!!</mark>

__load\_lss2\_clean__:
* dc: datacube.api.core.Datacube, the Datacube instance to load data with.
* products: list of products
* time: pair (list) of start and end dates (end date is not included (< instead of <=) !)
* lon: pair (list) of minimum and maximum longitude
* lat: pair (list) of minimum and maximum longitude
* measurements: list of measurements (without mask band, landsat and Sentinel 2 products prefix shouls be 'ls or 's2)
* resampl: (OPTIONAL) Up/Downsample ('up', 'down_mean', 'down_median' ) products and combine their output
* valid_cats:   (OPTIONAL) list of list of ints representing what category should be considered valid first Landsat categories, then Sentinel 2 categories

__updown\_sample__:
* ds_l: 'large' resolution xarray.Dataset (so far Landat product)
* ds_s: 'small' resolution xarray.Dataset (so far Sentinel 2 product)
* resampl: 'up' to upsample, 'down_mean' to downsample using mean values and 'down_median' to downsample using median values

Documentation for a given function can be accessed simply by adding ? at the end of the function in a cell. e.g. `load_lss2_clean?` or by selecting the function and pressing `Shift-Tab`.

In this demo Jupyter script, the user can either use the in-script function (below) or import it from ./swiss_utils/data_cube_utilities/sdc_utilities.py.

In [ ]:
# Make sure the script is using the proper kernel
try:
    %run ../swiss_utils/assert_env.py
except:
    %run ./swiss_utils/assert_env.py

In [ ]:
# Import modules

# reload module before executing code
%load_ext autoreload
%autoreload 2

# define modules locations (you might have to adapt define_mod_locs.py)
%run ../swiss_utils/define_mod_locs.py

# to plot figures
%matplotlib inline

import numpy as np
import xarray as xr

from datetime import datetime
from matplotlib import colors

from utils.data_cube_utilities.dc_mosaic import create_hdmedians_multiple_band_mosaic

from swiss_utils.data_cube_utilities.sdc_advutils import composite_fig, oneband_fig

import datacube
dc = datacube.Datacube()

# AND THE FUNCTION
from swiss_utils.data_cube_utilities.sdc_utilities import load_lss2_clean, updown_sample

As in the majority of demo_FUN_* notebook, the next cell contains the dataset configuration information:
- product
- geographical extent
- time period
- bands

But as the purpose of `mix_lss2` function is to mix Landsat and Sentinel 2 products, _product_ string, become _product**s**_ tupple.

Time period is also an issue as Sentinel 2 acquisition started in June 2015, start time should be more recent than this period.

If you are well aware of your ODC dataset and you understodd well what is written above, you can generate a configuration cell. If you aren't, it recommended to proceed as follows:
1. run the [config_tool](config_tool.ipynb) using **Sentinel 2** product and the _measurements_ containing at minimum `['blue', 'green', 'red', 'nir']` (as they are required for the current demo).
2. load the generated configuration cell in the next cell by running the magic `%load config_cell.txt`
3. use the _Select the product_ cell of [config_tool](config_tool.ipynb) again, to identity Landsat 7 and/or 8 product(s),
4. in the next cell rename _product_ variable to _product**s**_, use list instead of strings (e.g. in SDC context `products = ['s2_l2a_10m_swiss', 'ls7_ledaps_swiss', 'ls8_lasrc_swiss']`).

Comments:
- As mask band differ between Landsat and Sentinel 2, mask band do not need to be added in the measurements list. But in the case they are noth or only one added, the `mix_lss2` function will deal with this issue.
- As bands differ between Landsat and Sentinel 2, the `mix_lss2`function will only keep common bands.

In [ ]:
%load config_cell.txt

__Prepare a dataset combining Landsat and Sentinel 2 products by downsampling (decreasing the resolution) of Sentinel 2 into Landsat resolution (computing the median of 9 Sentinel 2 pixels)__

In [ ]:
dict_dsc, dict_cm = load_lss2_clean(dc = dc,                                    
                                    products = products ,
                                    time = [start_date, end_date],
                                    lon = (min_lon, max_lon),
                                    lat = (min_lat, max_lat),
                                    measurements = measurements,
                                    resampl = 'down_median')

__As both products are combined, the output consists in two dictionnaries containing a single dataset (dataset_clean and clean_mask)__

In [ ]:
# visualize the two dictionnaries

print(dict_dsc)
print(dict_cm)

In [ ]:
mosaic = create_hdmedians_multiple_band_mosaic(dict_dsc['lss2'],
                                               dict_cm['lss2'],
                                               operation='medoid')
composite_fig(mosaic,
              bands = ['red', 'green', 'blue'],
              title = 'Sentinel 2 and Landsat downsampled - median',
              scalebar_color = 'white',
              v_min = 0,
              v_max = 1500,
              max_size = 10)

__Repeat the process but this time by upsampling (increasing resolution) Landsat into Sentinel 2 resolution__

In [ ]:
dict_dsc, dict_cm = load_lss2_clean(dc = dc,                                    
                                    products = products ,
                                    time = [start_date, end_date],
                                    lon = (min_lon, max_lon),
                                    lat = (min_lat, max_lat),
                                    measurements = measurements,
                                    resampl = 'up')

In [ ]:
# Let's create and plot a composite mosaic

mosaic = create_hdmedians_multiple_band_mosaic(dict_dsc['lss2'],
                                               dict_cm['lss2'],
                                               operation='medoid')
composite_fig(mosaic,
              bands = ['red', 'green', 'blue'],
              title = 'Sentinel 2 and Landsat upsampled',
              scalebar_color = 'white',
              v_min = 0,
              v_max = 1500,
              max_size = 10)

__If the _`resampl`_ option is not used, the function will output two dictionnaries containing two dataset__

In [ ]:
dict_dsc, dict_cm = load_lss2_clean(dc = dc,                                    
                                    products = products ,
                                    time = [start_date, end_date],
                                    lon = (min_lon, max_lon),
                                    lat = (min_lat, max_lat),
                                    measurements = measurements)
print(dict_dsc)

In [ ]:
# Create and plot a Sentinel 2 composite mosaic

mosaic = create_hdmedians_multiple_band_mosaic(dict_dsc['s2'],
                                               dict_cm['s2'],
                                               operation='medoid')
composite_fig(mosaic,
              bands = ['red', 'green', 'blue'],
              title = 'Sentinel 2 only',
              scalebar_color = 'white',
              v_min = 0,
              v_max = 1500,
              max_size = 10)

__As upscaling increases the size of the large resolution dataset without increasing the level of information of the dataset, it is a waste of resources to derive products from upscaled dataset.__

__Let's compute NDVI of ls and s2 dataset and then combine them by upscaling__

In [ ]:
# compute NDVI
ndvi_s2 = (dict_dsc['s2'].nir - dict_dsc['s2'].red) / (dict_dsc['s2'].nir + dict_dsc['s2'].red)
ndvi_ls = (dict_dsc['ls'].nir - dict_dsc['ls'].red) / (dict_dsc['ls'].nir + dict_dsc['ls'].red)
# combine by upscaling
# ndvi dataArray need to be converted to dataset
ndvi_lss2 = updown_sample(ndvi_ls.to_dataset(name = 'ndvi'),
                          ndvi_s2.to_dataset(name = 'ndvi'),
                                             resampl = 'up')

__Finally lets compute ls, s2 and upscaled lss2 and visualize NDVI mean__

In [ ]:
# ls
ndvi_ls_mean = ndvi_ls.mean(dim=['time'])
oneband_fig(ndvi_ls_mean,
            leg = colors.LinearSegmentedColormap.from_list('ndvi', ['darkblue','blue','lightblue',
                                                                    'lightgreen','darkgreen'], N=256),
            title = 'Landsat only NDVI mean',
            scalebar_color = 'white',
            max_size = 10,
            v_min = -1,
            v_max = 1)

In [ ]:
# s2
ndvi_s2_mean = ndvi_s2.mean(dim=['time'])
oneband_fig(ndvi_s2_mean,
            leg = colors.LinearSegmentedColormap.from_list('ndvi', ['darkblue','blue','lightblue',
                                                                    'lightgreen','darkgreen'], N=256),
            title = 'Sentinel 2 only NDVI mean',
            scalebar_color = 'white',
            max_size = 10,
            v_min = -1,
            v_max = 1)

In [ ]:
# upscaled lss2
ndvi_lss2_mean = ndvi_lss2.mean(dim=['time'])
oneband_fig(ndvi_lss2_mean.ndvi,
            leg = colors.LinearSegmentedColormap.from_list('ndvi', ['darkblue','blue','lightblue',
                                                                    'lightgreen','darkgreen'], N=256),
            title = 'Sentinel 2 and Landsat upsampled NDVI mean',
            scalebar_color = 'white',
            max_size = 10,
            v_min = -1,
            v_max = 1)